Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [15]:
# this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)



In [16]:
num_steps = 801
# With gradient descent training, even
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 46.697090
Training accuracy: 7.8%
Validation accuracy: 11.7%
Loss at step 100: 11.604063
Training accuracy: 73.3%
Validation accuracy: 71.8%
Loss at step 200: 4.381855
Training accuracy: 78.9%
Validation accuracy: 76.3%
Loss at step 300: 1.937019
Training accuracy: 82.2%
Validation accuracy: 79.7%
Loss at step 400: 1.112057
Training accuracy: 83.9%
Validation accuracy: 81.2%
Loss at step 500: 0.829784
Training accuracy: 84.3%
Validation accuracy: 81.8%
Loss at step 600: 0.731848
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 700: 0.697519
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 800: 0.685387
Training accuracy: 84.8%
Validation accuracy: 82.1%
Test accuracy: 88.8%


In [18]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Hidden layer
  num_hidden_layer_features = 1024
  hidden_weights = tf.Variable(
  tf.truncated_normal([image_size * image_size, num_hidden_layer_features]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_layer_features]))
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
 
  # Output layer
  weights = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_output = tf.matmul(valid_relu, weights) + biases
  valid_prediction = tf.nn.softmax(valid_output)

  test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_output = tf.matmul(test_relu, weights) + biases
  test_prediction = tf.nn.softmax(test_output)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3480.373779
Minibatch accuracy: 5.5%
Validation accuracy: 35.1%
Minibatch loss at step 500: 21.178043
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.943275
Minibatch accuracy: 82.0%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 0.581393
Minibatch accuracy: 88.3%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.602335
Minibatch accuracy: 89.8%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.723866
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.762072
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Test accuracy: 90.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
batch_size = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Hidden layer
  num_hidden_layer_features = 1024
  hidden_weights = tf.Variable(
  tf.truncated_normal([image_size * image_size, num_hidden_layer_features]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_layer_features]))
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
 
  # Output layer
  weights = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_output = tf.matmul(valid_relu, weights) + biases
  valid_prediction = tf.nn.softmax(valid_output)

  test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_output = tf.matmul(test_relu, weights) + biases
  test_prediction = tf.nn.softmax(test_output)

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3582.684326
Minibatch accuracy: 20.0%
Validation accuracy: 17.0%
Minibatch loss at step 500: 88.339142
Minibatch accuracy: 50.0%
Validation accuracy: 63.3%
Minibatch loss at step 1000: 2.198003
Minibatch accuracy: 70.0%
Validation accuracy: 63.2%
Minibatch loss at step 1500: 1.261724
Minibatch accuracy: 80.0%
Validation accuracy: 66.6%
Minibatch loss at step 2000: 1.263242
Minibatch accuracy: 60.0%
Validation accuracy: 70.1%
Minibatch loss at step 2500: 1.847896
Minibatch accuracy: 60.0%
Validation accuracy: 71.1%
Minibatch loss at step 3000: 2.297153
Minibatch accuracy: 40.0%
Validation accuracy: 57.5%
Test accuracy: 63.1%


In [ ]:
The L2 loss actually increases due to the overfitting, and overall validation and test accuracy therefore goes down.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [65]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Hidden layer
  num_hidden_layer_features = 1024
  hidden_weights = tf.Variable(
  tf.truncated_normal([image_size * image_size, num_hidden_layer_features]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_layer_features]))
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  p_keep_hidden = tf.placeholder(tf.float32) 
  hidden_layer = tf.nn.dropout(hidden_layer, p_keep_hidden)
 
  # Output layer
  weights = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_output = tf.matmul(valid_relu, weights) + biases
  valid_prediction = tf.nn.softmax(valid_output)

  test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_output = tf.matmul(test_relu, weights) + biases
  test_prediction = tf.nn.softmax(test_output)

In [66]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, p_keep_hidden: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, p_keep_hidden: 1.0})
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 361.886902
Minibatch accuracy: 7.8%
Validation accuracy: 37.1%
Minibatch loss at step 500: 21.644703
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 13.610455
Minibatch accuracy: 78.1%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 6.559472
Minibatch accuracy: 88.3%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 1.928006
Minibatch accuracy: 89.8%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 2.513754
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 2.307645
Minibatch accuracy: 78.1%
Validation accuracy: 82.0%
Test accuracy: 89.5%


In [68]:
batch_size = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Hidden layer
  num_hidden_layer_features = 1024
  hidden_weights = tf.Variable(
  tf.truncated_normal([image_size * image_size, num_hidden_layer_features]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_layer_features]))
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  p_keep_hidden = tf.placeholder(tf.float32) 
  hidden_layer = tf.nn.dropout(hidden_layer, p_keep_hidden)
 
  # Output layer
  weights = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_output = tf.matmul(valid_relu, weights) + biases
  valid_prediction = tf.nn.softmax(valid_output)

  test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_output = tf.matmul(test_relu, weights) + biases
  test_prediction = tf.nn.softmax(test_output)

In [69]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, p_keep_hidden: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels, p_keep_hidden: 1.0})
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 473.950989
Minibatch accuracy: 0.0%
Validation accuracy: 18.7%
Minibatch loss at step 500: 9300.664062
Minibatch accuracy: 40.0%
Validation accuracy: 47.1%
Minibatch loss at step 1000: 107973.109375
Minibatch accuracy: 50.0%
Validation accuracy: 57.9%
Minibatch loss at step 1500: 3237219.750000
Minibatch accuracy: 50.0%
Validation accuracy: 45.7%
Minibatch loss at step 2000: 154045008.000000
Minibatch accuracy: 20.0%
Validation accuracy: 36.7%
Minibatch loss at step 2500: 43497728.000000
Minibatch accuracy: 50.0%
Validation accuracy: 41.0%
Minibatch loss at step 3000: 70685472.000000
Minibatch accuracy: 10.0%
Validation accuracy: 48.3%
Test accuracy: 52.0%


In [ ]:
In this case, with extreme overfitting the loss increased and accuracy decreased because we had limited data and we were losing half of our data.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [139]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer 
  num_hidden_layer_features = 1024
  hidden_weights = tf.Variable(
  tf.truncated_normal([image_size * image_size, num_hidden_layer_features]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_layer_features]))
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

  # Hidden layer 2
  num_hidden_layer_features_2 = 512
  hidden_weights_2 = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features, num_hidden_layer_features_2]))
  hidden_biases_2 = tf.Variable(tf.zeros([num_hidden_layer_features_2]))
  hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer, hidden_weights_2) + hidden_biases_2)
 
  # Output layer
  weights = tf.Variable(
  tf.truncated_normal([num_hidden_layer_features_2, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer_2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(hidden_weights) + 0.01*tf.nn.l2_loss(hidden_weights_2) 
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1500, 0.9, staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_relu_2 = tf.nn.relu(tf.matmul(valid_relu, hidden_weights_2) + hidden_biases_2)
  valid_output = tf.matmul(valid_relu_2, weights) + biases
  valid_prediction = tf.nn.softmax(valid_output)

  test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
  test_relu_2 = tf.nn.relu(tf.matmul(test_relu, hidden_weights_2) + hidden_biases_2)
  test_output = tf.matmul(test_relu_2, weights) + biases
  test_prediction = tf.nn.softmax(test_output)


In [140]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9900.409180
Minibatch accuracy: 8.6%
Validation accuracy: 25.2%
Minibatch loss at step 500: 3255.942383
Minibatch accuracy: 53.9%
Validation accuracy: 54.2%
Minibatch loss at step 1000: 1974.507446
Minibatch accuracy: 63.3%
Validation accuracy: 63.0%
Minibatch loss at step 1500: 1197.425293
Minibatch accuracy: 76.6%
Validation accuracy: 70.1%
Minibatch loss at step 2000: 763.471619
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 487.060760
Minibatch accuracy: 74.2%
Validation accuracy: 77.0%
Minibatch loss at step 3000: 310.767395
Minibatch accuracy: 78.1%
Validation accuracy: 79.2%
Minibatch loss at step 3500: 207.437836
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 4000: 138.465576
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 92.538773
Minibatch accuracy: 84.4%
Validation accuracy: 82.5%
Minibatch loss at step 5000: 64.429718
Minibatch accuracy: 8